<img src="https://www.politecnicos.com.br/img/075.jpg" alt="Grupo Turing" height="420" width="420">
Autor: Ariel Guerreiro

# Hyperparameter Tuning

## Introdução
Hiperparâmetros são parâmetros cujo valor é determinado antes da aprendizagem de um modelo. Para a biblioteca [sklearn](https://docs.google.com/document/u/1/d/1W7Z0SvUGHGU3YkTo1rECUMJervk7FhPlfCxfNkgiOdI/edit?usp=drive_open&ouid=110097037757552537186), os hiperparâmetros são fornecidos como argumentos da função responsável pela construção de um modelo, com cada modelo possuindo hiperparâmetros específicos.

Para obter um melhor modelo de IA, é interessante obter os melhores hiperparâmetros. Esse notebook busca mostrar duas formas de se obter os melhores hiperparâmetros, dado um modelo e um dataset


## Construindo um modelo
Antes da busca de hiperparâmetros, vamos primeiro criar o dataframe, separá-lo em treino e teste e criar um modelo. Neste notebook, será utilizado o [breast cancer wisconsin dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html), um dos datasets já presentes no sklearn, além da Decision Tree como modelo. Esse modelo já foi abordado em um **[Turing Talks](https://medium.com/turing-talks)**, cujo texto pode ser conferido [neste link](https://medium.com/turing-talks/turing-talks-17-modelos-de-predi%C3%A7%C3%A3o-decision-tree-610aa484cb05).

Para a execução deste processo, é necessária a importação das bibliotecas [pandas](https://drive.google.com/drive/u/1/folders/1FlpRFtXXMytmqjW_ZPVd_U76jwHwcYY1) e a partes da já mencionada scikit-learn

In [89]:
import pandas as pd
from sklearn import datasets

In [90]:
#Criação do dataframe
breast = datasets.load_breast_cancer()
df = pd.DataFrame(breast.data, columns=breast.feature_names)
df['target'] = pd.Series(breast.target)
df.head(10)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0
5,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,0.07613,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440,0
6,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,0.05742,...,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368,0
7,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,0.2196,0.07451,...,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510,0
8,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,0.07389,...,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720,0
9,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,0.2030,0.08243,...,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750,0


In [91]:
#Separando os dados em 'train' e 'test'
from sklearn.model_selection import train_test_split

X = df.drop('target', axis=1)
y = df.target

X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0,test_size=0.2) #reservamos 20% dos dados para teste

In [92]:
#Criação de um modelo Decision Tree
from sklearn import tree

clf = tree.DecisionTreeClassifier() # vamos utilizar os hiperparâmetros default inicialmente

Com isso feito, podemos começar a busca pelos hiperparâmetros ideais para estes dados.


## Busca pelos melhores hiperparâmetros

### 1º Método: Grid Search
Grid search consiste no teste de todas as combinações de hiperparâmetros, dentro dos limites que estabelecemos. Podemos ver os hiperparâmetros de um knn:

In [93]:
clf.get_params().keys() #Podemos ver os parâmetros que o modelo possui

dict_keys(['class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'presort', 'random_state', 'splitter'])

Queremos descobrir se estes são os valores ideais, e é neste momento que o grid search é útil. Primeiro, criaremos um dicionário que possui os nomes dos hiperparâmetros e os valores que queremos testar. Caso haja dúvida sobre o significado de cada um, o ideal é buscar na [documentação](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html). Para esse exemplo, testaremos somente alguns do hiperparâmetros do modelo, deixando os outros no padrão.

In [94]:
#criação do dicionário, com o auxilio da biblioteca NumPy
import numpy as np

max_depth = np.arange(1, 10)
min_samples_split = np.arange(2, 10)
min_samples_leaf = np.arange(1, 10)

param_grid = {'max_depth' : max_depth,
              'min_samples_split' : min_samples_split,
              'min_samples_leaf' : min_samples_leaf
}

param_grid

{'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
 'min_samples_split': array([2, 3, 4, 5, 6, 7, 8, 9]),
 'min_samples_leaf': array([1, 2, 3, 4, 5, 6, 7, 8, 9])}

Com o dicionário criado, podemos começar o teste de todas as permutações possíveis dentro do determinado em 'param_grid'. Importaremos então o Grid Search do sklearn, usando como critério de avaliação 'scoring = accuracy'. Para obtenção deste valor, utilizaremos [validação cruzada](https://scikit-learn.org/stable/modules/cross_validation.html), separando o dataframe de treino em 10 partes (cv = 10). Além desses argumentos, também são passados o modelo e o dicionário de valores possíveis

In [95]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(clf, param_grid, cv=10, scoring='accuracy')

grid.fit(X_train, y_train)

C:\Users\Ariel\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                         'min_samples_leaf': array([1, 2, 3, 

Podemos então utilizar o próprio 'grid' para a previsão, que já se utiliza dos melhores hiperparâmetros encontrados nos limites estabelecidos. É possível visualizar os hiperparâmetros encontrados utilizando-se do argumento **.best_params_**. 

In [96]:
grid.best_params_

{'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 8}

Para comparar, vamos criar um modelo com os hiperparâmetros padrão e compará-lo com o 'grid', utilizando-se do **accuracy_score** do sklearn para obter os resultados.

In [100]:
#Avaliação dos modelos
from sklearn.metrics import accuracy_score

#modelo com GridSearch
y_pred_grid = grid.predict(X_test)
accuracy_grid= accuracy_score(y_test, y_pred_grid)

#modelo com hiperparâmetros padrão
y_pred_std = tree_std.predict(X_test)
accuracy_std = accuracy_score(y_test, y_pred_std)

In [101]:
print("Modelo com hiperparâmetros: {}".format(accuracy_grid))
print("Modelo padrão: {}".format(accuracy_std))

Modelo com hiperparâmetros: 0.9473684210526315
Modelo padrão: 0.9122807017543859


Como podemos observar, os hiperparâmetros aumentaram a acurácia do modelo em relação ao modelo padrão.

Porém, este método de busca, testando todas as permutações possíveis, tende a demorar muito para ser executado, à medida que o número de hiperparâmetros testados e suas possibilidades aumenta. Neste exemplo, onde foram testados somente 3 hiperparâmetros, o número de combinações que tiveram que ser testadas foi de 9 x 9 x 8 = **648 combinações**, o que já é demorado para se executar. Para modelos mais complexos, o número de combinações cresce muito além disso, tornando este método de busca de hiperparâmetros inviável. Para ajudar neste problema, surge outro método:


### 2º Método: Random Search
O Random Search se assimila ao Grid Search, mas, ao invés de serem testadas todas as permutações possíveis de forma exaustiva, são executadas um número determinado de combinações. O tradeoff existente entre os métodos é de que o Random Search é mais rápido de ser executado, pelo menor número de testes, mas não garante que a combinação de hiperparâmetros é a ideal.

Para o Random Search, usaremos o mesmo dicionário 'param_grid' utilizado no Grid Search, porém, é possível aumentar os limites dos valores possíveis para os hiperparâmetros, já que não serão testadas todas as possibilidades

In [102]:
#Criação de uma nova decision tree (essa etapa é desnecessária neste caso)
clf = tree.DecisionTreeClassifier()

In [103]:
from sklearn.model_selection import RandomizedSearchCV

#É possível perceber a semelhança com o GridSearchCV
random = RandomizedSearchCV(clf, param_grid, cv=10, scoring='accuracy', n_iter=300)

random.fit(X_train, y_train)

C:\Users\Ariel\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=DecisionTreeClassifier(class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort=False,
                                                    random_state=None,
                                                    splitter='best'

In [104]:
# Podemos ver os parâmetros encontrados e avaliar o score do modelo
print(random.best_params_)

y_pred_rand = random.predict(X_test)
accuracy_rand = accuracy_score(y_test, y_pred_rand)

{'min_samples_split': 5, 'min_samples_leaf': 1, 'max_depth': 8}


Podemos então comparar os 3 resultados, com os dois métodos utilizados:

In [106]:
print("Modelo com GridSearchCV: {}".format(accuracy_grid))
print("Modelo com RandomizedSearchCV: {}".format(accuracy_rand))
print("Modelo padrão: {}".format(accuracy_std))

Modelo com GridSearchCV: 0.9473684210526315
Modelo com RandomizedSearchCV: 0.9385964912280702
Modelo padrão: 0.9122807017543859


## Conclusão
É possível observar que ambas as formas de hyperparameter tuning foram capazes de gerar hiperparâmetros melhores que os obtidos pelo padrão do modelo, mas o 'score' do modelo que se utilizou do Random Search não obteve o resultado do Grid Search, pois o teste de forma aleatória não garante que os melhores hiperparâmetros serão encontrados, o que possui maior chance de ser encontrado pelo Grid Search.

Em uma situação real, com datasets e modelos mais complexos, o ideal é se executar um Random Search com possibilidades mais abrangentes e realizar um Grid Search com valores próximos aos encontrados pelo Random Search